In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/sample_submission.csv
/kaggle/input/test.csv.zip
/kaggle/input/train.csv.zip
/kaggle/input/description.md
/kaggle/input/sample_submission.csv.zip
/kaggle/input/train.csv
/kaggle/input/test.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/description.md
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv


In [3]:
import re
import nltk
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize.regexp import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression


In [4]:
stop_words = stopwords.words('english')


In [5]:
pd.options.display.max_columns = None


In [6]:
df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')


In [7]:
# оставляю только буквы и цифры
df['clean_text'] = df.comment_text.apply(
    lambda row: re.sub(r'[^a-z0-9]+', ' ', row.lower()).strip()
)


In [8]:
stemmer = SnowballStemmer("english")
tokenizer = RegexpTokenizer(r'\w{2,}')

def preprocessing(text):
    new_words = tokenizer.tokenize(text)
    new_list = []
    for w in new_words:
        if w not in stop_words:
            w = stemmer.stem(w)
            new_list.append(w)
    new_list = ' '.join(new_list)
    return new_list


In [9]:
corpus = [preprocessing(text) for text in tqdm(df.clean_text)]


  0%|          | 0/159571 [00:00<?, ?it/s]

In [10]:
df['corpus'] = corpus


In [11]:
df.sample(5)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text,corpus
120012,81c07aa77e12ede8,ASSHOLE\n\nYou are an asshole,1,0,1,0,1,0,asshole you are an asshole,asshol asshol
20788,36e10faef714ce52,"""\n What about """"List of specific collective n...",0,0,0,0,0,0,what about list of specific collective nouns b...,list specif collect noun collect term neverthe...
115203,67ffab17160ffce5,"""\n\n Captain symbol? \n\nCan I as why you are...",0,0,0,0,0,0,captain symbol can i as why you are adding the...,captain symbol ad bullet new york citi subway ...
93863,fafb4834f9872104,Image:Louraine_solo.jpg listed for deletion \n...,0,0,0,0,0,0,image louraine solo jpg listed for deletion an...,imag lourain solo jpg list delet imag media fi...
23036,3ce6e2aa99c0fec4,"""\nTry reading the cites before commenting aga...",0,0,0,0,0,0,try reading the cites before commenting again ...,tri read cite comment cheer


In [12]:
# features = corpus
# target_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult']


In [13]:
# for col in target_cols:
#     df.loc[df[col] == 1, 'target'] = col

# df['target'].fillna(0, inplace=True)


In [14]:
X_train, X_test = train_test_split(
    df['corpus'], test_size = 0.4, random_state = 2024
) 

X_valid, X_test = train_test_split(
    X_test, test_size = 0.5, random_state = 2024
) 

print('train size', X_valid.shape)
print('test size', X_test.shape)
print('train size', X_train.shape)


train size (31914,)
test size (31915,)
train size (95742,)


In [15]:
vec = TfidfVectorizer(stop_words=stop_words)
features_train = vec.fit_transform(X_train)
features_test = vec.transform(X_test)
features_valid = vec.transform(X_valid)


In [16]:
target_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [17]:
for col in target_cols:
    check_proportion_df = pd.concat([
    df[col].value_counts(normalize=True), 
        df.loc[X_train.index, col].value_counts(normalize=True),
        df.loc[X_test.index, col].value_counts(normalize=True)
    ], axis=1)

    check_proportion_df.columns = ['full', 'train', 'test']
    
    print(col)
    display(check_proportion_df)
    print('\n'*2)


toxic


,full,train,test
toxic,,,
0,0.904156,0.904525,0.903776
1,0.095844,0.095475,0.096224





severe_toxic


,full,train,test
severe_toxic,,,
0,0.990004,0.989597,0.991133
1,0.009996,0.010403,0.008867





obscene


...

,full,train,test
threat,,,
0,0.997004,0.997086,0.996647
1,0.002996,0.002914,0.003353





insult


,full,train,test
insult,,,
0,0.950636,0.950607,0.952279
1,0.049364,0.049393,0.047721





identity_hate


,full,train,test
identity_hate,,,
0,0.991195,0.990892,0.992229
1,0.008805,0.009108,0.007771


In [18]:
models_list = []
for col in tqdm(target_cols):
    clf = LogisticRegression(class_weight='balanced', n_jobs=-1)
    clf.fit(features_train, df.loc[X_train.index, col])
    
    models_list += [(col, clf)]


  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
for item in models_list:
    col, clf = item
    print(col)
    print(f'train f1: {f1_score(df.loc[X_train.index, col], clf.predict(features_train))}')
    print(f'test f1: {f1_score(df.loc[X_test.index, col], clf.predict(features_test))}')
    print(f'valid f1: {f1_score(df.loc[X_valid.index, col], clf.predict(features_valid))}' + '\n'*2)


toxic
train f1: 0.8213588623478018
test f1: 0.7353932584269662
valid f1: 0.7359100491918482


severe_toxic
train f1: 0.5180754226267881
test f1: 0.382522671063479
valid f1: 0.39969372128637054


obscene
train f1: 0.856252106504887
test f1: 0.7581246670218433
valid f1: 0.7826086956521738


threat
train f1: 0.4765157984628523
test f1: 0.30068337129840544
valid f1: 0.33250620347394544


insult
train f1: 0.7641179816364669
test f1: 0.6501910828025478
valid f1: 0.6841085271317829


identity_hate
train f1: 0.4916737228337567
test f1: 0.32928942807625644
valid f1: 0.372093023255814




In [20]:
full_features = vec.fit_transform(df['corpus'])

models_list = []
for col in tqdm(target_cols):
    clf = LogisticRegression(class_weight='balanced', n_jobs=-1)
    clf.fit(full_features, df[col])
    models_list += [(col, clf)]


  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [21]:
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')


In [22]:
# оставляю только буквы и цифры
test['clean_text'] = test.comment_text.apply(
    lambda row: re.sub(r'[^a-z0-9]+', ' ', row.lower()).strip()
)


In [23]:
test['corpus'] = [preprocessing(text) for text in tqdm(test.clean_text)]


  0%|          | 0/153164 [00:00<?, ?it/s]

In [24]:
corpus = vec.transform(test['corpus'])


In [25]:
for item in models_list:
    col, clf = item
    test[col] = clf.predict_proba(corpus)[:, 1]


In [26]:
test.sample(10)


,id,comment_text,clean_text,corpus,toxic,severe_toxic,obscene,threat,insult,identity_hate
2763,04c63df6f3a16b9b,""" \n\n :I removed that statement because """"con...",i removed that statement because controversy i...,remov statement controversi aganist bcs suppor...,0.066282,0.006079,0.029837,0.001773,0.016586,0.014439
11258,12e2ca6eeedadd6c,== i think your a fag == \n\n like seriously,i think your a fag like seriously,think fag like serious,0.989404,0.273784,0.942277,0.013498,0.967470,0.917379
104920,af24b44a89333b0d,Robert James Duquette III is the definition of...,robert james duquette iii is the definition of...,robert jame duquett iii definit chao,0.044016,0.014245,0.068751,0.019838,0.058098,0.015774
130376,d9e1b14342d738cb,""" \n\n == Lazerhawk is not on Wikipedia??? == ...",lazerhawk is not on wikipedia that is fascinat...,lazerhawk wikipedia fascin total puzzl idiot f...,0.995251,0.040787,0.985563,0.005978,0.949776,0.209835
117004,c33b264757d28902,""" \n\n == Afrofuturism == \n\n What are the so...",afrofuturism what are the sources for this sec...,afrofutur sourc section seem like one person o...,0.010843,0.001487,0.015051,0.001625,0.008178,0.002910
88474,939437850140231e,You cannot block me you chode! You'll feel th...,you cannot block me you chode you ll feel the ...,cannot block chode feel fin barracuda,0.340124,0.017512,0.161288,0.029023,0.073884,0.023811
124599,d024b631d1b36c74,""" \n\n Answer me this and I'm not one to ever ...",answer me this and i m not one to ever attack ...,answer one ever attack user kind twat add dele...,0.018541,0.000558,0.011346,0.001260,0.012091,0.003069
8182,0db89d5961d43c30,(the Kiss me Stupid event),the kiss me stupid event,kiss stupid event,0.999593,0.334829,0.960368,0.045145,0.990453,0.263549
65860,6da3aa53fe4c78f4,== Userboxes == \n\n Am I allowed to post user...,userboxes am i allowed to post userboxes descr...,userbox allow post userbox describ made curren...,0.028793,0.009674,0.027723,0.002629,0.021399,0.001802
39336,4145b5b91c8c7859,"::Quack, do you actually know what OR means?",quack do you actually know what or means,quack actual know mean,0.217075,0.012934,0.080314,0.009956,0.178717,0.005869


In [27]:
test[['id'] + target_cols].to_csv('submission_combined.csv', index=False)
